In [ ]:
!pip install openai

In [ ]:
!pip install mysql-connector-python

In [ ]:
# package

import openai
import mysql.connector
from pprint import pprint # chatGPT message가 잘 출력되되도록 사용
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

In [ ]:
# Google Drive 마운트
# 마운트는 본인 드라이브만 가능하기에 공유 폴더 같은 경우 본인 드라이브에 바로가기를 추가하여 사용

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# API KEY : contact 계정의 API key 발급하여 사용

openai.api_key = ""

In [ ]:
today = datetime.today().strftime("%Y%m%d")
Path(f"/content/drive/MyDrive/HAI research/output/{today}/py_output").mkdir(parents=True, exist_ok=True)

In [ ]:
# google sheet를 바로 연동하는 것은 원본 훼손 위험이 존재하기에 우선 xlsx 파일을 다운로드 및 업로드하여여 활용

raw_data = pd.read_excel("/content/drive/MyDrive/HAI research/data/PoC 평가.xlsx", sheet_name='total')

In [ ]:
en_data = raw_data.copy()

In [ ]:
# PoC 평가가 A+인 인원들만 불러오기
# url 주소에 userID가 존재하는 형식 데이터만 불러오기
en_data = en_data.loc[en_data['총평 \n(Letter grade, like A+, B+)'] == 'A+']
en_data = en_data[en_data['framing docs'].str.contains('&userID', na = False)].reset_index(drop=True)

In [ ]:
# '&' 기준으로 파싱하여 user_id 리스트에 저장하기
id_list = []
for i in range(len(en_data)) :
  id_list.append(en_data['framing docs'][i].split('&')[1][7:])

In [ ]:
# DB 연동

mydb = mysql.connector.connect(
  host="",
  user="",
  password="",
  database=""
)

# 저장된 userID 정보 불러와서 사용
userCode = id_list[10]

mycursor = mydb.cursor()

mycursor.execute("SELECT user_id from userCodeTable where code='" + userCode + "'")

myresult = mycursor.fetchall()

userID = myresult[0][0]

mycursor.execute("SELECT user_id, question_id, response FROM responseTable where user_id='" + str(userID) + "'")

myresult = mycursor.fetchall()

userResponse = {}

# 0부터 시작되는 문항번호를 1부터 시작하도록 변경
for x in myresult:
    userResponse[x[1]+1] = x[2]

In [ ]:
userResponse

In [ ]:
# 문항 정보 입력
qna = pd.read_csv("/content/drive/MyDrive/HAI research/data/PoC_Question.csv")

In [ ]:
# 사례탐색 프롬프트
'''
output = []

# userResponse[8] = objective

for i in range(10) :

  messages = [{"role": "system", "content": "너는 AI 분야의 data scientist야"},
              {"role" : "system", "content" : "키워드를 제시할 경우, 인공지능의 영역이 아닌 제시된 주제와 관련된 도메인 영역의 키워드만 알려줘"},
              {"role" : "system", "content" : "키워드를 제시할 경우, '한국어(영어)'와 같은 형식으로 키워드를 알려줘."},
              {"role" : "system", "content" : "키워드는 포괄적인 단어가 아닌 구체적인 단어로 제시해줘."},
              {"role" : "user", "content" : f"인공지능의 목적은 {userResponse[8]}이야. 해당 주제와 연관성이 높은 다른 주제에 대한 키워드를 5개만 제시해줘. 키워드를 제시할 경우 인공지능과 연관된 키워드는 제시하지 말아줘."}
              ]

  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
  temperature=1,
  max_tokens=3000
  )

  chat_response = completion.choices[0].message.content
  output.append(chat_response)
'''

In [ ]:
# 검증/평가
'''
output = []

# userResponse[8] = objective

for i in range(5) :

  messages = [{"role": "system", "content": "너는 AI 분야의 data scientist야"},
              {"role" : "system", "content" : f"인공지능의 목적은 {userResponse[8]}이야."},
              {"role" : "system", "content" : "비즈니스 영역에 대해 이야기하지 않았다면 적합하지 않은 답변이야."},
              {"role" : "user", "content" : f"비즈니스 목표 측면에서 인공지능의 성공/실패를 판단할 수 있는 정량적인 기준은 {userResponse[9]}야. 해당 답변이 정량적이고 적절하게 답변했는지 '예/아니오'로 대답하고 그 이유를 알려줘."}
              ]

  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
  temperature=1,
  max_tokens=3000
  )

  chat_response = completion.choices[0].message.content
  output.append(chat_response)
'''

In [ ]:
# 시각화

output = []

# userResponse[8] = objective

for i in range(5) :

  messages = [{"role": "system", "content": "너는 AI 분야의 data scientist야"},
              {"role" : "system", "content" : "시각화 위젯은 다음 5가지 ['bar chart', 'line chart', 'scatter plot', 'box plot', 'heatmap'] 그래프 중에서만 추천해줘"},
              {"role" : "system", "content" : "시각화 위젯을 위젯을 추천할 시, 간단한 설명 및 주요 사용처를 설명해줘."},
              {"role" : "user", "content" : f"인공지능의 목적은 {userResponse[8]}야. 해당 인공지능의 목적에 적합한 시각화 위젯을 3개만 추천해줘."}
              ]

  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
  temperature=1,
  max_tokens=3000
  )

  chat_response = completion.choices[0].message.content
  output.append(chat_response)


In [ ]:
for i in range(5) :
  pprint(output[i])

In [ ]:
# import json

In [ ]:
for i in range(len(output)) :
  output[i] = output[i].split('\n')

In [ ]:
file_path = f"/content/drive/MyDrive/HAI research/output/{today}/py_output/{userCode}.json"
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(json.dumps(output, ensure_ascii=False, indent=2))

In [ ]:
def PoC_CoT_test(object:str, data, ChatGPT_api_key:str, data_path:str, model:str="gpt-3.5-turbo")->str:

    openai.api_key = ChatGPT_api_key

    messages = [
        {
            "role": "system",
            "content": f"AI의 주제는 '{object}'이야",
        },

         {
            "role": "system",
            "content": "임의의 데이터를 생성할 때, 샘플 데이터를 참고해줘.",
        },

        {
            "role": "system",
            "content":"샘플 데이터를 참고하여 python script를 사용하여 dataframe을 생성해줘",
        },

        {
            "role": "system",
            "content": "결과 전달 시 python script만 출력해줘",
        },

        {
            "role": "system",
            "content": "dataframe 생성 시 row가 100개가 되도록 임의의 데이터셋을 생성해줘.",
        },

        {
            "role": "system",
            "content": "data 값을 생성 시, random 함수를 사용해줘.",
        },

        {
            "role": "system",
            "content": "임의로 생성하는 dataframe의 value값들은 feature의 특성에 맞게 생성해줘",
        },

         {
            "role": "system",
            "content": f"생성한 dataframe의 저장 경로는 {data_path}야",
        },
         {
            "role": "system",
            "content": f"dataframe은 csv 파일 형태로 저장해주고 저장하는 파일의 명칭은 {userCode}이야",
        },
         {
            "role": "system",
            "content": "위 단계들을 차근차근 생각해줘",
        },
        {
            "role": "user",
            "content": f"샘플 데이터 세트는 '{data}'이야. 이를 참고하여 dataframe을 생성하는 python script를 만들어줘"
        }
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages, temperature=0.5)
    answer = response["choices"][0]["message"]["content"]
    return answer
'''
    messages = [
        {
            "role": "system",
            "content": f"지금부터 {answer}를 리뷰할거야",
        },
        {
            "role": "system",
            "content": "차근차근 생각하고 효율적으로 코드를 구축해줘",
        },

        {
            "role": "system",
            "content": "결과 전달 시 python script만 출력해줘",
        },

         {
            "role": "system",
            "content":f"생성한 {answer}가 데이터프레임을 지정된 경로에 잘 생성하는지 체크해줘",
        },

         {
            "role": "system",
            "content":f"생성한 {answer}에 {data}을 참고하여 feature와 Label 이름이 잘 부여되었는지 체크해줘",
        },

        {
            "role": "system",
            "content": f"실행하는데 문제가 발생하지 않을 경우 {answer} python script를 그대로 출력해줘",
        },

        {
            "role": "user",
            "content": f'생성한 {answer}를 리뷰해주고 python script 결과만 다시 전달해줘'
        }
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages)
    answer = response["choices"][0]["message"]["content"]
'''

In [ ]:
target = '타이타닉 생존자 예측'
columns = ['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
values = [[1,0,3,"Braund, Mr. Owen Harris",'male',22,1,0,'A/5 21171',7.25,'C23','S'],
 [2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",'female',38,1,0,'PC 17599',71.2833,'C85','C'],
 [3,1,3,"Heikkinen, Miss. Laina",'female',26,0,0,'STON/O2. 3101282',7.925,'C24','S']]
data = pd.DataFrame(values, columns=columns)

In [ ]:
output = PoC_CoT_test(target, data, ChatGPT_api_key=openai.api_key, data_path=f'/content/drive/MyDrive/HAI research/output/{today}/py_output', model="gpt-3.5-turbo")

In [ ]:
# 일단 대충 얼기설기 짜놓았습니다.
# 앞에서부터 탐색 / 뒤에서부터 탐색 두개로 각각 발견되면 멈추게 짜면 될 것 같습니다.
divided = output.split('\n')
idx = []
for i, w in enumerate(divided):
    if w.startswith("```") or w.endswith("```"):
        idx.append(i)
output_code = '\n'.join(divided[idx[0]+1:idx[1]])

pprint(output_code)

In [ ]:
with open(f'/content/drive/MyDrive/HAI research/output/{today}/output_poc_{userCode}_v9.py', 'w') as f:
    f.write(output_code)